### Install dependencies

In [51]:
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Load files


In [52]:
import pandas as pd

def load_and_process(req_path, test_path):
    req_df = pd.read_csv(req_path, sep=',', on_bad_lines='skip')
    test_df = pd.read_csv(test_path, sep=',', on_bad_lines='skip')
    
    # Some Purpose columns are intentionally left blank for now; populate them with empty strings
    test_df['Purpose'] = test_df['Purpose'].fillna('')

    req_text_fields = ['Feature', 'Description']
    test_text_fields = ['Purpose', 'Test steps']

    # Combine text columns for similarity matching
    req_df['full_text'] = req_df[req_text_fields].astype(str).agg(' '.join, axis=1)
    test_df['full_text'] = test_df[test_text_fields].astype(str).agg(' '.join, axis=1)
    
    # Convert to lists
    req_texts = req_df['full_text'].tolist()
    test_texts = test_df['full_text'].tolist()

    return req_df, test_df, req_texts, test_texts



### Cosine Similarity

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_cosine(req_df, test_df, req_texts, test_texts):
    vectorizer = TfidfVectorizer()
    documents = req_texts + test_texts
    tfidf_matrix = vectorizer.fit_transform(documents)

    similarity_matrix = cosine_similarity(tfidf_matrix[:len(req_texts)], tfidf_matrix[len(req_texts):])

    # TODO: tweak and see if it can perform better
    threshold = 0.38

    # Find matches for each requirement
    matches = {}
    for i, req_id in enumerate(req_df['ID']):
        matched_test_ids = []
        for j, similarity in enumerate(similarity_matrix[i]):
            if similarity >= threshold:
                matched_test_ids.append(str(test_df.iloc[j]['ID']))  # ensure string for JSON

        matches[str(req_id)] = matched_test_ids

    # Results
    return matches


### Calculate accuracy

In [54]:
def calculate_accuracy(pred_dict, ground_truth_path):
    
    # Load and clean ground truth
    gt_df = pd.read_csv(ground_truth_path).dropna(subset=['Req ID'])
    
    # Convert ground truth data into a dictionary
    gt_dict = {
        row['Req ID']: list(map(str, str(row['Test ID']).split(','))) if pd.notna(row['Test ID']) else []
        for _, row in gt_df.iterrows()
    }
    pred_dict = dict(pred_dict)

    correct = 0
    total = 0

    # Compare predicted and ground truth
    for req_id in set(pred_dict.keys()) | set(gt_dict.keys()):
        predicted = set(pred_dict.get(req_id, []))
        actual = set(gt_dict.get(req_id, []))
        total += 1
        if predicted == actual:
            correct += 1
    # Calculate accuracy
    accuracy = (correct / total) * 100 if total > 0 else 0

    print("*********************************************")
    print(f"Accuracy: {accuracy:.2f}% ({correct}/{total} correct)")
    print("*********************************************")
    return accuracy


### Display predicted and Ground Truth results side by side

In [55]:
def print_side_by_side(pred_dict, mapping_path):
    
    # Ground Truth Dataset
    gt_df = pd.read_csv(mapping_path)

    # Convert to dict
    gt_dict = {
        row['Req ID']: list(map(str, str(row['Test ID']).split(','))) if pd.notna(row['Test ID']) else []
        for _, row in gt_df.iterrows()
    }
    
    # Get the union of all requirement IDs
    all_req_ids = sorted(set(pred_dict.keys()) | set(gt_dict.keys()))

    print(f"\n{'Req ID':<10} | {'Predicted':<20} | {'Ground Truth'}")
    print("-" * 55)

    for req_id in all_req_ids:
        pred_ids = ', '.join(map(str, pred_dict.get(req_id, [])))
        gt_ids = ', '.join(gt_dict.get(req_id, []))
        print(f"{req_id:<10} | {pred_ids:<20} | {gt_ids}")


In [56]:
import json 
import os
import time

# Datasets folders 
datasets = ['BTHS', 'ENCO', 'SnakeGame', 'Mozilla', 'HealthWatcher', 'SourceTracker']
#Initialize summary table for visualization of results
summary_table = pd.DataFrame(columns=["Dataset", "Accuracy"])

# For every dataset find the following:
for dataset in datasets:
    
    req_path = f'../data/{dataset}/RE.csv'
    test_path = f'../data/{dataset}/ST.csv'
    ground_truth_mapping_path = f'../data/{dataset}/mapping.csv'

    print(f"Dataset: {dataset}")
    
    # Load and process data
    req_df, test_df, req_texts, test_texts = load_and_process(req_path, test_path)
    
    # Find cosine similarity and measure execution time
    start = time.time()
    predicted = compute_cosine(req_df, test_df, req_texts, test_texts)
    t = time.time() - start

    # Payload is formatted exactly as at REST-AT and saved to the corresponding data folder
    # Cosine similarity result is deterministic, so there's no need for multiple executions
    payload: dict[str, dict] = {
        "meta": {
            "req_path": req_path,
            "test_path": test_path,
            "mapping_path": ground_truth_mapping_path
        },
        "data": {
            "links": predicted,
            "time-to-analyze": t
        }
    }
    log_dir: str = f"./{dataset}"
    os.makedirs(log_dir, exist_ok=True)

    with open(f"{log_dir}/res.json", "w+") as out:
        json.dump(payload, out, indent=2)


    # ***************************************************
    # Analyzing results (accuracy + visual comparison)
    # ***************************************************

    # Print results side by side 
    print_side_by_side(predicted, ground_truth_mapping_path)
    # Compute accuracy
    accuracy = calculate_accuracy(predicted, ground_truth_mapping_path)

    summary_table = pd.concat([
        summary_table,
        pd.DataFrame([{"Dataset": dataset, "Accuracy": accuracy}])
    ], ignore_index=True)

    print("\n")


# Print Final Results Table
print("+------------------+---------------+")
print("| Dataset          | Accuracy (%)  |")
print("+------------------+---------------+")
for _, row in summary_table.iterrows():
    accuracy_str = f"{row['Accuracy']:.2f}%"
    print(f"| {row['Dataset']:<16} | {accuracy_str:<13.7} |") 
print("+------------------+---------------+")


Dataset: BTHS

Req ID     | Predicted            | Ground Truth
-------------------------------------------------------
4.2        |                      | HSP/AG/IAC/BV-01-I,  HSP/HS/IAC/BV-01-I, HSP/AG/IAC/BV-02-I, HSP/HS/IAC/BV-02-I, HSP/AG/OAC/BV-01-I, HSP/HS/OAC/BV-01-I
4.2.1      | HSP/HS/IAC/BV-01-I, HSP/AG/IAC/BV-02-I, HSP/HS/IAC/BV-02-I | HSP/HS/IAC/BV-01-I, HSP/AG/IAC/BV-02-I, HSP/HS/IAC/BV-02-I
4.2.2      |                      | 
4.3        | HSP/AG/IAC/BV-01-I, HSP/HS/IAC/BV-01-I, HSP/HS/IAC/BV-02-I, HSP/AG/OAC/BV-01-I, HSP/HS/OAC/BV-01-I, HSP/HS/ACR/BV-01-I, HSP/HS/ACR/BV-02-I, HSP/AG/ACT/BV-01-I, HSP/AG/ACT/BV-02-I | HSP/AG/OAC/BV-01-I, HSP/HS/OAC/BV-01-I
4.5        | HSP/AG/IAC/BV-01-I, HSP/HS/IAC/BV-01-I, HSP/AG/IAC/BV-02-I, HSP/HS/IAC/BV-02-I, HSP/AG/OAC/BV-01-I, HSP/HS/OAC/BV-01-I, HSP/AG/ACR/BV-01-I, HSP/HS/ACR/BV-01-I, HSP/HS/ACR/BV-02-I, HSP/AG/ACT/BV-01-I, HSP/AG/ACT/BV-02-I, HSP/HS/ACT/BV-02-I | HSP/AG/ACR/BV-01-I, HSP/HS/ACR/BV-01-I, HSP/HS/ACR/BV-02-I
4.6.1   

/var/folders/q8/fj2nlt_d3vqd2yr3m8kvphq80000gn/T/ipykernel_78730/2972789516.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_table = pd.concat([
